In [11]:
import pandas as pd
import numpy as np

input_file = '../assets/data/recommendation_dataset.csv'

dfp = pd.read_csv(input_file, sep=';', skiprows=1)
dfp = dfp.iloc[:, 1:].replace(99.0, None)

user_coef = np.zeros((len(dfp), len(dfp.columns) - 1, len(dfp.columns) - 1))

for row in dfp.iterrows():
	user = row[0]
	row = row[1]
	for i in range(1, len(row)):
		if row[i] is not None:
			for j in range(1, len(row)):
				if row[j] is not None:
					if i == j:
						user_coef[user][i - 1][j - 1] = 1
					elif row[i] == row[j]:
						user_coef[user][i - 1][j - 1] = 1
					else:
						user_coef[user][i - 1][j - 1] = 1 / abs(row[i] - row[j])
	print("Done with user", user)

print(user_coef[0])



C:\Users\Thomas Chourret\AppData\Local\Temp\ipykernel_16828\4192914305.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[i] is not None:
C:\Users\Thomas Chourret\AppData\Local\Temp\ipykernel_16828\4192914305.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[j] is not None:
C:\Users\Thomas Chourret\AppData\Local\Temp\ipykernel_16828\4192914305.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif row[i] == row

Done with user 0
Done with user 1
Done with user 2
Done with user 3
Done with user 4
Done with user 5
Done with user 6
Done with user 7
Done with user 8
Done with user 9
Done with user 10
Done with user 11
Done with user 12
Done with user 13
Done with user 14
Done with user 15
Done with user 16
Done with user 17
Done with user 18
Done with user 19
Done with user 20
Done with user 21
Done with user 22
Done with user 23
Done with user 24
Done with user 25
Done with user 26
Done with user 27
Done with user 28
Done with user 29
Done with user 30
Done with user 31
Done with user 32
Done with user 33
Done with user 34
Done with user 35
Done with user 36
Done with user 37
Done with user 38
Done with user 39
Done with user 40
Done with user 41
Done with user 42
Done with user 43
Done with user 44
Done with user 45


KeyboardInterrupt: 